In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import random
from utils import render_single, render_multiple, get_dataset_files, extract_random_entries, generate_pixel_columns
from IPython.display import display, Image as IPImage

In [ ]:
#files = get_dataset_files()
#files = random.choices(files, k=10)
files = ['../dataset/line.ndjson', '../dataset/mountain.ndjson', '../dataset/yoga.ndjson', '../dataset/bee.ndjson', '../dataset/binoculars.ndjson', '../dataset/snowman.ndjson', '../dataset/squiggle.ndjson', '../dataset/church.ndjson', '../dataset/guitar.ndjson', '../dataset/house.ndjson']
data = [extract_random_entries(file, 10000, recognized=True) for file in files]
flat_data = [item for sublist in data for item in sublist]
df = pd.DataFrame.from_dict(flat_data, orient='columns')
word = flat_data[0]['word']
print(f'Loaded {len(df)} entries from {files}')

In [ ]:
img = random.choice(flat_data)['drawing']
display(IPImage(render_single(img)))

In [ ]:
imgs = df.sample(1000)['drawing']
display(IPImage(render_multiple(imgs)))

In [ ]:
df2 = df.sample(len(df))
train_amt = int(len(df2) * .80)

train = df2[:train_amt]
test = df2[train_amt:]

train = train.reset_index()
test = test.reset_index()

train_processed = generate_pixel_columns(train, resolution=28, invert_color=True).drop(columns=['countrycode', 'timestamp', 'recognized', 'key_id', 'drawing'])
test_processed = generate_pixel_columns(test, resolution=28, invert_color=True)
print(f'Train: {len(train_processed)} entries, test: {len(test_processed)} entries.')
print(train_processed.sample(1))

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
import time
import joblib

y = train_processed['word']
X = train_processed.drop(columns=['word'])

model = OneVsRestClassifier(LinearSVC(random_state=0, max_iter=10000, dual=False), n_jobs=-1).fit(X, y)
filename = '../model' + str(int(time.time()))
filedata = joblib.dump(model, filename)

In [ ]:
sample = test_processed.sample(1)
sample_predict = sample.drop(columns=['countrycode', 'timestamp', 'recognized', 'key_id', 'drawing', 'word'])

prediction = model.predict(sample_predict)
display(IPImage(render_single(sample['drawing'].iloc[0])))
print(prediction[0])
print(f"{sample['word'].iloc[0]} == {prediction[0]} ? {sample['word'].iloc[0] == prediction[0]}")